In [ ]:
! pip install -q transformers

     |████████████████████████████████| 3.4 MB 18.0 MB/s 
     |████████████████████████████████| 596 kB 49.6 MB/s 
     |████████████████████████████████| 895 kB 38.0 MB/s 
     |████████████████████████████████| 61 kB 347 kB/s 
     |████████████████████████████████| 3.3 MB 40.3 MB/s 


In [ ]:
import lzma
from os import listdir
from os.path import isfile, join, splitext
import sys
from transformers import BertTokenizer, BertModel
import numpy as np
from tqdm import tqdm
import os, psutil
import gc
import torch
import pickle
import pdb
from collections import OrderedDict
from operator import getitem
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

In [ ]:
pp=0
class corpus:
  def __init__(self, corpus_path, max_snt=None, max_len=None, min_len=None):
    self.corpus_path = corpus_path
    self.sentence_num = 0
    self.max_sentence = max_snt
    self.max_len = max_len
    self.min_len = min_len

  def __iter__(self):
    fp = open(self.corpus_path)
    for snt in fp:
      snt = snt.split()
      if (self.max_len is None or len(snt) < self.max_len) and (self.min_len is None or len(snt) >= self.min_len):
        self.sentence_num += 1
      else:
        continue
      if (self.max_sentence is None or self.sentence_num <= self.max_sentence):
        yield ' '.join(snt)
      else:
        fp.close()
        return
    fp.close()


def get_token_vecs(batch, layer):
  token_ids = tokenizer(batch, return_tensors="pt", padding=True, truncation=True) #['input_ids']
  with torch.no_grad():
    token_vecs = bert(**token_ids, output_hidden_states=True).hidden_states[:25]
    print(len(token_vecs))
    out = np.concatenate(list(map(lambda x: x[0].numpy(), token_vecs)))
  token_batch = []
  token_vector = []
  for j,sentence in enumerate(token_ids['input_ids']):
    tokens = tokenizer.convert_ids_to_tokens(sentence)
    vectors = token_vecs[j].detach().numpy()
    i = 0
    while tokens[i] != '[SEP]' and i < len(tokens): # why are [CLS] tokens arent ignored ?
      w = tokens[i]
      v = vectors[:,i,:]
      while tokens[i+1].startswith('##'):
        i += 1
        w += tokens[i][2:]
        v += vectors[:,i,:]

      token_batch.append(w)
      token_vector.append(v)
      #yield (w,v.tolist())
      i += 1
  return zip(token_batch, token_vector)

def update_token_vecs(batch, layer):
  for (t,v) in get_token_vecs(batch, layer):
    try:
      vv = np.expand_dims(v, axis=1)
      words_stats[t]['n'] += 1
      #words_stats[t]['S'] += vv
      #words_stats[t]['XXT'] += vv.dot(vv.T)
      func_AccumulateVectors (word=t,output_dir=output_dir,F=1,S=vv,flag=False)
      print(t,":",words_stats[t]['n'])
    except KeyError:
      continue

def load_WSD_data_file(file_path):
  word_sense_dict = {}
  with open(file_path) as fp:
    for line in fp:
      toks = line.strip().split()
      if len(toks) < 2: continue
      for tok in toks[1:]:
        (word, sense) = tok.split('%')
        try:
          word_sense_dict[word].add(sense)
        except KeyError:
          word_sense_dict[word] = set([sense])
  return word_sense_dict

def load_vocabs(vocab_file):
  with open(vocab_file) as fp:
    vocabs = [line.strip() for line in fp]
  return vocabs
 


def func_AccumulateVectors (word,output_dir,F,S,flag=False):
  if flag:
      output_path = '{0}/{1}.pkl'.format(output_dir, word)
      try:
        word_vectors[word]
      except: 
        word_vectors[word] = {'counter':0,'freq':0}   

      try:
        fp = open(output_path, 'ab') 
        #pickle.dump(filter(lambda i: i[0] not in ['counter','freq'], word_vectors[word].items()),fp)
        pickle.dump(word_vectors[word], fp)
        fp.close()
      except:
          fp = open(output_path, 'w') 
          pickle.dump(word_vectors[word], fp)
          fp.close()

      word_vectors[word] = {'counter':0,'freq':word_vectors[word]['freq'] }
  else:
    try:
      word_vectors[word]
    except:
      word_vectors[word] = {'counter':0,'freq':0}

    word_vectors[word]['Vectors_{}'.format(word_vectors[word]['freq']+1)]=S
    word_vectors[word]['counter']+=1
    word_vectors[word]['freq']+=1
    if (word_vectors[word]['counter'] >=F):
        output_path = '{0}/{1}.pkl'.format(output_dir, word)
        try:
          fp = open(output_path, 'ab') 
          #pickle.dump(filter(lambda i: i[0] not in ['counter','freq'], word_vectors[word].items()),fp)
          pickle.dump(word_vectors[word], fp)
          fp.close()
        except:
            fp = open(output_path, 'w') 
            pickle.dump(word_vectors[word], fp)
            fp.close()

        word_vectors[word] = {'counter':0,'freq':word_vectors[word]['freq']}

12474 iteration, each iteration had 25 batches, executed for 3hrs 41 mins

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
bert = BertModel.from_pretrained('bert-large-uncased')
bert_hsize = bert.config.hidden_size

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.25G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#if __name__ == '__main__':
corpus_path = '/content/drive/MyDrive/Research Project/random-word-vectors-main/60000semcor+omsti.data.snt'#sys.argv[1]
file_extension = ['.xz'] #sys.argv[2:]

In [ ]:
import pandas as pd
WSD_data_file = '/content/drive/MyDrive/Research Project/random-word-vectors-main/Words.csv'
df = pd.read_csv (WSD_data_file)
words = list(pd.DataFrame(df,columns=['Word']).values.flatten())
print(words)
#print(words=='human')

['all', 'security', 'national', 'state', 'work', 'new', 'law', 'support', 'report', 'one', 'well', 'social', 'special', 'time', 'country', 'government', 'number', 'system', 'order', 'world', 'resolution', 'there', 'use', 'action', 'only', 'community', 'process', 'need', 'first', 'peace', 'economic', 'level', 'program', 'take', 'about', 'right', 'up', 'made', 'education', 'so', 'further', 'political', 'party', 'respect', 'part', 'trade', 'protection', 'court', 'high', 'make', 'see', 'case', 'draft', 'act', 'regard', 'view', 'but', 'effective', 'session', 'capacity', 'progress', 'review', 'president', 'continue', 'conflict', 'conference', 'area', 'policy', 'do', 'representative', 'service', 'issue', 'future', 'force', 'space', 'set', 'increase', 'mission', 'important', 'term', 'period', 'situation', 'family', 'full', 'plan', 'present', 'place', 'access', 'long', 'end', 'even', 'approach', 'major', 'now', 'violence', 'consideration', 'local', 'care', 'concern', 'control', 'like', 'project

In [ ]:
len(words)

20

In [ ]:
#vocab_file = sys.argv[2]
max_vocabs = 1100
#words = load_vocabs(vocab_file)
#words = words[0:min(max_vocabs, len(words))]
#words = words[-15:]
output_dir = '/content/drive/MyDrive/Research Project/random-word-vectors-main/TM_Project'#sys.argv[3]

layer = int(4)
assert abs(layer) < bert.config.num_hidden_layers

word_vectors = {}
words_stats = {}
for w in words:
  words_stats[w] = {'XXT':np.zeros((bert_hsize, bert_hsize), dtype='float32'), 'S':np.zeros((bert_hsize,1)), 'n':0.0}

In [ ]:
bsize =25
batch = []
i = 0
max_snt = None
min_len = None
max_len = None

with tqdm(total=max_snt) as pbar:
  for tokens in corpus(corpus_path, max_snt=max_snt, max_len=max_len, min_len=min_len):

    #print("A",tokens,"\n")
    batch += [tokens]
    #print("B",batch,"\n")

    #print("c",len(batch),"\n")
    if len(batch) == bsize:
      #t = get_token_vecs(batch, layer)
      #print("C",t,"\n")
      update_token_vecs(batch, layer)
      batch.clear()

    process = psutil.Process(os.getpid())
    #print(process)
    mem = process.memory_info().rss
    pbar.set_description("{0:1.2f}M".format(mem/1024/1024))
    pbar.update(1)

  
  if batch:
    update_token_vecs(batch, layer)
    #t = get_token_vecs(batch, layer)
    #print("D",t,"\n")
    batch.clear()

4260.71M: : 48it [00:27,  1.35s/it]

25
would : 1.0


4260.78M: : 73it [00:50,  1.09s/it]

25
would : 2.0
space : 1.0
would : 3.0
know : 1.0


4260.78M: : 98it [01:10,  1.06it/s]

...

25
support : 922.0
would : 1470.0
support : 923.0


4588.72M: : 21823it [5:59:40,  1.02it/s]

25
support : 924.0
would : 1471.0
support : 925.0


4588.72M: : 21848it [6:00:05,  1.02it/s]

25
would : 1472.0
island : 66.0
space : 332.0
space : 333.0
space : 334.0
space : 335.0
space : 336.0


In [ ]:
def get_statistics(word):
  n = words_stats[word]['n']
  s = words_stats[word]['S']
  #print(s.shape)
  xxt = words_stats[word]['XXT'] 
  #print(xxt.shape)
  cov = xxt/(n-1) - s.dot(s.T)/((n-1)**2)
  mu = s/n
  u,s,vh = np.linalg.svd(cov)
  logdet = np.log(s[s>1e-10]).sum()

  return cov, mu, logdet

In [ ]:
statistics_dict = {}
mean = []
cov = []
cnt = 0

for w in words:
  #func_AccumulateVectors (w,output_dir,F=25,S=[],flag=True)
  
  output_path = '{0}/word_stats/{1}_{2}.pkl'.format(output_dir, w, layer)
  fp = open(output_path, 'wb') 
  pickle.dump(words_stats[w], fp)
  fp.close()

In [ ]:
tt = OrderedDict(sorted(words_stats.items(),
       key = lambda x: getitem(x[1], 'n'), reverse=True))

In [ ]:
mean = []
cov = []
word_key = tt.keys()
#cnt = 0
max_pc = 0
with open (output_dir+'PCA_Analysis.txt','w')as file:
  for w in word_key:
    if tt[w]['n'] >1:
      #cnt +=1
      cov, mu, logdet = get_statistics(w)
      pca = PCA()
      pca.fit(cov)
      var = np.round(pca.explained_variance_ratio_*100,decimals=1)
      cmsm=np.cumsum(var)
      if max_pc ==0:
        pc = np.where(cmsm<=95)
        max_pc = len(pc[0])
        
      if len(pc[0])==0:
        pc = [[0]]

      labels = ['PC'+ str(x) for x in range(1,len(pc[0])+1)]
      s = "Word: {}\n".format(w)
      s+="Number of Occurance: {}\n".format(words_stats[w]['n'])
      s+="PC Variance %: {}\n".format(var[pc[0]])
      s+="Cumulative Sum: {}\n".format(cmsm[pc[0]])
      s+="PC: {}\n".format(pc[0])
      s+="Principle Components: {}\n\n".format(labels)
      file.write(s)
